In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

import warnings 
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('AIQ.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       1093 non-null   float64
 1   TM      1093 non-null   float64
 2   Tm      1093 non-null   float64
 3   SLP     1093 non-null   float64
 4   H       1093 non-null   float64
 5   VV      1093 non-null   float64
 6   V       1093 non-null   float64
 7   VM      1093 non-null   float64
 8   PM 2.5  1092 non-null   float64
dtypes: float64(9)
memory usage: 77.0 KB


In [4]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [5]:
# <<--- Independent and Dependent Variables --->>

X = df.iloc[:, :-1]
Y = df['PM 2.5']

In [6]:
# <<-- Train Test Split -->>

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

In [7]:
# <<-- Scale Data -->>

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

## Model Building by Hyperparameter Tuning

In [8]:
def build_model(hp):
    model = keras.Sequential()
    
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
        
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
            loss='mean_absolute_error',
            metrics=['mean_absolute_error'])
    
    return model

In [9]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='Models')

In [10]:
'''
max_trials represents the number of hyperparameter combinations that will be tested by the tuner, while execution_per_trial is
the number of models that should be built and fit for each trial for robustness purposes.

For example, let's imagine you have a shallow network (one hidden layer) with the following parameter search space:

Number of Hidden units: 16, 32, 48, 64
Choice of activation functions: ReLu, Sigmoid, TanH
You have a total of 12 combinations (4 times 3).

Then, at this point, when you set up max_trials = 40, you configure the tuner to find 40 random tuples of the 
Hidden units and activation functions, e.g. (16, ReLu), (32, Sigmoid), (16, TanH)... up until 40 tuples. For each tuple you 
will run as many executions as you also set up in execution_per_trial variable, given that depending on how the model runs the
optimization process, final results could be very different. For each trial and execution, the tuner will fit the model with 
as many epochs as you configure in the script.
'''
print()

In [11]:
tuner.search_space_summary()

In [12]:
tuner.search(x_train, y_train,
             epochs=100,
             validation_data=(x_test, y_test))

Epoch 1/100
28/28 [==============================] - ETA: 0s - loss: 114.3863 - mean_absolute_error: 114.386 - ETA: 0s - loss: nan - mean_absolute_error: nan          - 0s 18ms/step - loss: nan - mean_absolute_error: nan - val_loss: 95.6578 - val_mean_absolute_error: 95.6578
Epoch 2/100
28/28 [==============================] - ETA: 0s - loss: 99.0629 - mean_absolute_error: 99.062 - ETA: 0s - loss: 85.8861 - mean_absolute_error: 85.886 - 0s 10ms/step - loss: nan - mean_absolute_error: nan - val_loss: 49.1108 - val_mean_absolute_error: 49.1108
Epoch 3/100
28/28 [==============================] - ETA: 0s - loss: 37.2408 - mean_absolute_error: 37.240 - ETA: 0s - loss: 47.3637 - mean_absolute_error: 47.363 - 0s 11ms/step - loss: nan - mean_absolute_error: nan - val_loss: 42.8954 - val_mean_absolute_error: 42.8954
Epoch 4/100
28/28 [==============================] - ETA: 0s - loss: 39.1970 - mean_absolute_error: 39.197 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 10ms/step - 

28/28 [==============================] - ETA: 0s - loss: 29.0301 - mean_absolute_error: 29.030 - ETA: 0s - loss: 33.9982 - mean_absolute_error: 33.998 - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 35.0280 - val_mean_absolute_error: 35.0280
Epoch 32/100
28/28 [==============================] - ETA: 0s - loss: 38.5534 - mean_absolute_error: 38.553 - ETA: 0s - loss: 32.7767 - mean_absolute_error: 32.776 - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 34.7386 - val_mean_absolute_error: 34.7386
Epoch 33/100
28/28 [==============================] - ETA: 0s - loss: 30.9135 - mean_absolute_error: 30.913 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 9ms/step - loss: nan - mean_absolute_error: nan - val_loss: 33.4599 - val_mean_absolute_error: 33.4599
Epoch 34/100
28/28 [==============================] - ETA: 0s - loss: 37.1600 - mean_absolute_error: 37.160 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_

28/28 [==============================] - ETA: 0s - loss: 35.2069 - mean_absolute_error: 35.206 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 32.6615 - val_mean_absolute_error: 32.6615
Epoch 62/100
28/28 [==============================] - ETA: 0s - loss: 25.7697 - mean_absolute_error: 25.769 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 34.2068 - val_mean_absolute_error: 34.2068
Epoch 63/100
28/28 [==============================] - ETA: 0s - loss: 29.6686 - mean_absolute_error: 29.668 - ETA: 0s - loss: 30.6232 - mean_absolute_error: 30.623 - 0s 9ms/step - loss: nan - mean_absolute_error: nan - val_loss: 31.2575 - val_mean_absolute_error: 31.2575
Epoch 64/100
28/28 [==============================] - ETA: 0s - loss: 27.3130 - mean_absolute_error: 27.313 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_

28/28 [==============================] - ETA: 0s - loss: 30.0175 - mean_absolute_error: 30.017 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 31.2547 - val_mean_absolute_error: 31.2547
Epoch 92/100
28/28 [==============================] - ETA: 0s - loss: 17.8306 - mean_absolute_error: 17.830 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 31.7757 - val_mean_absolute_error: 31.7757
Epoch 93/100
28/28 [==============================] - ETA: 0s - loss: 18.3196 - mean_absolute_error: 18.319 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 33.6159 - val_mean_absolute_error: 33.6159
Epoch 94/100
28/28 [==============================] - ETA: 0s - loss: 19.8140 - mean_absolute_error: 19.814 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_

28/28 [==============================] - ETA: 0s - loss: 41.5616 - mean_absolute_error: 41.561 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 35.3908 - val_mean_absolute_error: 35.3908
Epoch 22/100
28/28 [==============================] - ETA: 0s - loss: 31.4665 - mean_absolute_error: 31.466 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 35.2611 - val_mean_absolute_error: 35.2611
Epoch 23/100
28/28 [==============================] - ETA: 0s - loss: 27.2499 - mean_absolute_error: 27.249 - ETA: 0s - loss: 35.9983 - mean_absolute_error: 35.998 - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 36.8404 - val_mean_absolute_error: 36.8404
Epoch 24/100
28/28 [==============================] - ETA: 0s - loss: 38.2997 - mean_absolute_error: 38.299 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 5ms/step - loss: nan - mean_

...

28/28 [==============================] - ETA: 0s - loss: 55.3897 - mean_absolute_error: 55.389 - ETA: 0s - loss: 70.3698 - mean_absolute_error: 70.369 - ETA: 0s - loss: 65.0036 - mean_absolute_error: 65.003 - ETA: 0s - loss: 65.7101 - mean_absolute_error: 65.710 - ETA: 0s - loss: 63.6113 - mean_absolute_error: 63.611 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 16ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.0021 - val_mean_absolute_error: 64.0021
Epoch 51/100
28/28 [==============================] - ETA: 0s - loss: 62.0123 - mean_absolute_error: 62.012 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 16ms/step - loss: nan - mean_absolute_error: nan - val_loss

28/28 [==============================] - ETA: 0s - loss: 53.5336 - mean_absolute_error: 53.533 - ETA: 0s - loss: 66.6932 - mean_absolute_error: 66.693 - ETA: 0s - loss: 66.5327 - mean_absolute_error: 66.532 - ETA: 0s - loss: 65.7906 - mean_absolute_error: 65.790 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 16ms/step - loss: nan - mean_absolute_error: nan - val_loss: 63.7336 - val_mean_absolute_error: 63.7336
Epoch 66/100
28/28 [==============================] - ETA: 0s - loss: 74.0413 - mean_absolute_error: 74.041 - ETA: 0s - loss: 63.6365 - mean_absolute_error: 63.636 - ETA: 0s - loss: 68.4538 - mean_absolute_error: 68.453 - ETA: 0s - loss: 70.5209 - mean_absolute_error: 70.520 - ETA: 0s - loss: 68.1901 - mean_absolute_error: 68.190 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 15ms/step - loss: nan - mean_absolu

28/28 [==============================] - ETA: 0s - loss: 76.4438 - mean_absolute_error: 76.443 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 17ms/step - loss: nan - mean_absolute_error: nan - val_loss: 67.8786 - val_mean_absolute_error: 67.8786
Epoch 81/100
28/28 [==============================] - ETA: 0s - loss: 80.8531 - mean_absolute_error: 80.853 - ETA: 0s - loss: 67.9172 - mean_absolute_error: 67.917 - ETA: 0s - loss: 68.8895 - mean_absolute_error: 68.889 - ETA: 0s - loss: 66.8233 - mean_absolute_error: 66.823 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 16ms/step - loss: n

Epoch 95/100
28/28 [==============================] - ETA: 0s - loss: 64.1862 - mean_absolute_error: 64.186 - ETA: 0s - loss: 65.8140 - mean_absolute_error: 65.814 - ETA: 0s - loss: 66.2438 - mean_absolute_error: 66.243 - ETA: 0s - loss: 66.4600 - mean_absolute_error: 66.460 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 14ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.1479 - val_mean_absolute_error: 64.1479
Epoch 96/100
28/28 [==============================] - ETA: 0s - loss: 81.4204 - mean_absolute_error: 81.420 - ETA: 0s - loss: 71.7339 - mean_absolute_error: 71.733 - ETA: 0s - loss: 67.7903 - mean_absolute_error: 67.790 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 17ms/step - loss: nan - mean_absolute_error: nan - val_loss: 63.6046 - val_mean_absolu

INFO:tensorflow:Oracle triggered exit


In [13]:
model = tuner.get_best_models(num_models=2)[0] # Best model

In [14]:
tuner.results_summary()

In [15]:
model

In [16]:
# Evaluate the model

loss, accuracy = model.evaluate(x_test, y_test) # Test the accuracy on test data

7/7 [==============================] - ETA: 0s - loss: 21.0879 - mean_absolute_error: 21.087 - 0s 3ms/step - loss: 22.5668 - mean_absolute_error: 22.5668


In [17]:
print(loss, accuracy)

22.566816329956055 22.566816329956055


In [18]:
model.metrics_names

['loss', 'mean_absolute_error']

## Save Model

In [19]:
model.save('Model.h5')

## Single Prediction

In [20]:
T = 7.4
TM = 9.8
Tm = 4.8
SLP = 1017.6
H = 93.0
VV = 0.5
V = 4.3
VM = 9.4

data = ss.transform([[T, TM, Tm, SLP, H, VV, V, VM]])

In [21]:
data

array([[-2.57159622, -3.4210371 , -1.96667138,  1.25630887,  1.90387122,
        -2.05010366, -0.62192   , -0.89038409]])

In [22]:
model.predict(data)

array([[209.89272]], dtype=float32)